## **About Aweshome**

Aweshome is a chatbot created by implementing machine learning in it. Its manufacture is carried out as an additional feature that will complement the smart home application created that also called Aweshome.
This application has a motto:

***Aweshome: Make your home smart and awesome ***

The chatbot feature will help users explicitly manage the appliances in their house, as well as to know the condition of their house.

## **Steps to create Aweshome Chatbot**

The following is an overview of the steps taken to create an Aweshome chatbot:
1. Prepare the required packages, including the Deep Learning, Tensorflow, Keras, Pickle, and NLTK (Natural Language Processing Toolkit) libraries.
2. Prepare a dataset in the form of a collection of user input and output that must be displayed in response to the Aweshome chatbot. The file is saved in JSON type and is named "intents."
3. Perform data preparation including import required packages, load the JSON file and extract the required data.
4. Creating models including training and testing models.
5. Integrating Aweshome chatbot with application.




In [22]:
#importing the libraries
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [23]:
#importing the dataset
with open('sample_data/intents.json') as file:
    data = json.load(file)

In [24]:
#getting all the data to lists
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [25]:
#
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [26]:
#
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [27]:
#printing the data
data

{'intents': [{'patterns': ['Hi',
    'Halo',
    'Hello',
    'Helo',
    'Apa kabar',
    'Hey',
    'Hola',
    'Hello',
    'Hai',
    'Tes',
    'Cek'],
   'responses': ['Hello, ada yang bisa dibantu?',
    'Senang bertemu Anda kembali :)',
    'Halo, katakan apa yang Anda inginkan!',
    'Hai, apakah Anda perlu bantuan?',
    'Hai, beri tahu apa yang bisa saya bantu!',
    'Hai, ada yang bisa saya bantu?',
    'Katakan apa yang Anda inginkan saat ini!',
    'Senang rasanya kita dapat berjumpa kembali',
    'Hai, alhamdulillah ya kita bisa bercakap bersama'],
   'tag': 'salam1'},
  {'patterns': ["Assalamu'alaikum",
    'Assalamualaikum',
    "Assalamu'alaikum Warahmatullahi Wabarakatuh",
    "Assalamu'alaikum Warohmatulloh",
    'Assalamualaikum Warahmatullah'],
   'responses': ["Wa'alaikumsalam Warahmatullahi Wabarakatuh.",
    "Wa'alaikumsalam",
    "Wa'alaikumsalam Warohmatullahi Wabarokatuh"],
   'tag': 'salam2'},
  {'patterns': ['Siapa kamu?',
    'Siapa namamu?',
    'Ini den

In [28]:
#creating the model

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))



In [29]:
#compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
#view model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 15)                255       
                                                                 
Total params: 16,799
Trainable params: 16,799
Non-trainable params: 0
____________________________________________________

In [31]:
#training the model
epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/550
8/8 [==============================] - 1s 3ms/step - loss: 2.7049 - accuracy: 0.1050
Epoch 2/550
8/8 [==============================] - 0s 3ms/step - loss: 2.6974 - accuracy: 0.1050
Epoch 3/550
8/8 [==============================] - 0s 4ms/step - loss: 2.6884 - accuracy: 0.1050
Epoch 4/550
8/8 [==============================] - 0s 4ms/step - loss: 2.6780 - accuracy: 0.1176
Epoch 5/550
8/8 [==============================] - 0s 3ms/step - loss: 2.6672 - accuracy: 0.1050
Epoch 6/550
8/8 [==============================] - 0s 3ms/step - loss: 2.6539 - accuracy: 0.1008
Epoch 7/550
8/8 [==============================] - 0s 3ms/step - loss: 2.6394 - accuracy: 0.1050
Epoch 8/550
8/8 [==============================] - 0s 3ms/step - loss: 2.6240 - accuracy: 0.1218
Epoch 9/550
8/8 [==============================] - 0s 4ms/step - loss: 2.6072 - accuracy: 0.1092
Epoch 10/550
8/8 [==============================] - 0s 3ms/step - loss: 2.5893 - accuracy: 0.1218
Epoch 11/550
8/8 [===========

In [32]:
# saving model
model.save("chat_model")

import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    

# saving label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets
